# 🌍 Unified Global Weather Model
## Single MLP for All Countries - Temperature Trend Forecasting

---

**Objectives:**
1. Clean data (fix country names)
2. Feature engineering (temporal, cyclical, geographic, lag features)
3. Build and train MLP Neural Network in PyTorch
4. Hyperparameter tuning with Optuna
5. Evaluate and save model artifacts

**Input:** Country + Date → **Output:** 7-day Temperature Forecast

---

## 1. Setup & Libraries

In [1]:
# Core Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Machine Learning
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Hyperparameter Tuning
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Utilities
import joblib
import json
import os
from tqdm import tqdm

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Libraries loaded")
print(f"🖥️ Device: {device}")
print(f"🔥 PyTorch version: {torch.__version__}")

✅ Libraries loaded
🖥️ Device: cuda
🔥 PyTorch version: 2.9.1+cu130


---
## 2. Load & Clean Data

In [2]:
# Load raw data
df = pd.read_csv('../data/raw/GlobalWeatherRepository.csv')
print(f"📊 Raw data: {len(df):,} rows, {df['country'].nunique()} countries")

📊 Raw data: 114,203 rows, 211 countries


In [3]:
# Country name fixes (from data quality analysis)
COUNTRY_FIXES = {
    'Inde': 'India',
    'Mexique': 'Mexico',
    'Bélgica': 'Belgium',
    'Estonie': 'Estonia',
    'Letonia': 'Latvia',
    'Jemen': 'Yemen',
    'Malásia': 'Malaysia',
    'Marrocos': 'Morocco',
    'Polônia': 'Poland',
    'Saudi Arabien': 'Saudi Arabia',
    'Südkorea': 'South Korea',
    'Turkménistan': 'Turkmenistan',
    'Saint-Vincent-et-les-Grenadines': 'Saint Vincent and the Grenadines',
    'USA United States of America': 'United States of America',
    'Kyrghyzstan': 'Kyrgyzstan',
    'Macedonia': 'North Macedonia',
    'Seychelles Islands': 'Seychelles',
    'Fiji Islands': 'Fiji',
    'Swaziland': 'Eswatini',
    'Brunei Darussalam': 'Brunei',
    'Timor-Leste': 'East Timor',
    "Cote d'Ivoire": 'Ivory Coast',
    'Democratic Republic of Congo': 'DR Congo',
    "Lao People's Democratic Republic": 'Laos',
    # Non-Latin scripts - map to correct English names
    'Гватемала': 'Guatemala',
    'Польша': 'Poland',
    'Турция': 'Turkey',
    'كولومبيا': 'Colombia',
    '火鸡': 'Turkey',  # Chinese for Turkey
    'Komoren': 'Comoros',
}

# Apply fixes
df['country'] = df['country'].replace(COUNTRY_FIXES)
print(f"✅ Applied {len(COUNTRY_FIXES)} country name fixes")

✅ Applied 30 country name fixes


In [4]:
# Parse dates
df['last_updated'] = pd.to_datetime(df['last_updated'])
df['date'] = df['last_updated'].dt.date
df['date'] = pd.to_datetime(df['date'])

print(f"📅 Date range: {df['date'].min()} to {df['date'].max()}")

📅 Date range: 2024-05-16 00:00:00 to 2025-12-24 00:00:00


In [5]:
# Aggregate to daily data per country
daily_df = df.groupby(['country', 'date']).agg({
    'temperature_celsius': 'mean',
    'humidity': 'mean',
    'pressure_mb': 'mean',
    'wind_kph': 'mean',
    'precip_mm': 'sum',
    'cloud': 'mean',
    'uv_index': 'mean',
    'latitude': 'first',
    'longitude': 'first'
}).reset_index()

print(f"📊 Daily aggregated data: {len(daily_df):,} rows")
print(f"🌍 Countries: {daily_df['country'].nunique()}")

📊 Daily aggregated data: 108,238 rows
🌍 Countries: 191


In [6]:
# Filter countries with sufficient data (>= 30 days for lag features)
MIN_DAYS = 30
country_days = daily_df.groupby('country').size()
valid_countries = country_days[country_days >= MIN_DAYS].index.tolist()

daily_df = daily_df[daily_df['country'].isin(valid_countries)].copy()

print(f"✅ Filtered to {len(valid_countries)} countries with >= {MIN_DAYS} days")
print(f"📊 Final daily data: {len(daily_df):,} rows")

✅ Filtered to 186 countries with >= 30 days
📊 Final daily data: 108,232 rows


---
## 3. Feature Engineering

In [7]:
# Sort by country and date for lag calculations
daily_df = daily_df.sort_values(['country', 'date']).reset_index(drop=True)

# Temporal Features
daily_df['month'] = daily_df['date'].dt.month
daily_df['day_of_month'] = daily_df['date'].dt.day
daily_df['day_of_week'] = daily_df['date'].dt.dayofweek
daily_df['day_of_year'] = daily_df['date'].dt.dayofyear
daily_df['week_of_year'] = daily_df['date'].dt.isocalendar().week.astype(int)
daily_df['quarter'] = daily_df['date'].dt.quarter
daily_df['is_weekend'] = (daily_df['day_of_week'] >= 5).astype(int)

print("✅ Temporal features created")

✅ Temporal features created


In [8]:
# Cyclical Encoding (captures periodicity)
daily_df['month_sin'] = np.sin(2 * np.pi * daily_df['month'] / 12)
daily_df['month_cos'] = np.cos(2 * np.pi * daily_df['month'] / 12)
daily_df['day_sin'] = np.sin(2 * np.pi * daily_df['day_of_week'] / 7)
daily_df['day_cos'] = np.cos(2 * np.pi * daily_df['day_of_week'] / 7)
daily_df['day_of_year_sin'] = np.sin(2 * np.pi * daily_df['day_of_year'] / 365)
daily_df['day_of_year_cos'] = np.cos(2 * np.pi * daily_df['day_of_year'] / 365)

print("✅ Cyclical features created")

✅ Cyclical features created


In [9]:
# Lag Features (per country)
LAG_DAYS = [1, 2, 3, 7, 14, 30]

for lag in LAG_DAYS:
    daily_df[f'temp_lag_{lag}'] = daily_df.groupby('country')['temperature_celsius'].shift(lag)

# Rolling Statistics (per country)
daily_df['temp_rolling_mean_7'] = daily_df.groupby('country')['temperature_celsius'].transform(
    lambda x: x.shift(1).rolling(window=7, min_periods=1).mean()
)
daily_df['temp_rolling_mean_14'] = daily_df.groupby('country')['temperature_celsius'].transform(
    lambda x: x.shift(1).rolling(window=14, min_periods=1).mean()
)
daily_df['temp_rolling_std_7'] = daily_df.groupby('country')['temperature_celsius'].transform(
    lambda x: x.shift(1).rolling(window=7, min_periods=1).std()
)

print("✅ Lag and rolling features created")

✅ Lag and rolling features created


In [10]:
# Encode Country
country_encoder = LabelEncoder()
daily_df['country_encoded'] = country_encoder.fit_transform(daily_df['country'])

print(f"✅ Encoded {len(country_encoder.classes_)} countries")

✅ Encoded 186 countries


In [11]:
# Drop rows with NaN (from lag features)
initial_len = len(daily_df)
daily_df = daily_df.dropna().reset_index(drop=True)
print(f"📊 Dropped {initial_len - len(daily_df):,} rows with NaN")
print(f"📊 Final dataset: {len(daily_df):,} rows")

📊 Dropped 5,580 rows with NaN
📊 Final dataset: 102,652 rows


In [12]:
# Define feature columns
FEATURE_COLS = [
    # Country encoding
    'country_encoded',
    # Geographic
    'latitude', 'longitude',
    # Temporal
    'month', 'day_of_month', 'day_of_week', 'day_of_year', 'quarter', 'is_weekend',
    # Cyclical
    'month_sin', 'month_cos', 'day_sin', 'day_cos', 'day_of_year_sin', 'day_of_year_cos',
    # Lag features
    'temp_lag_1', 'temp_lag_2', 'temp_lag_3', 'temp_lag_7', 'temp_lag_14', 'temp_lag_30',
    # Rolling stats
    'temp_rolling_mean_7', 'temp_rolling_mean_14', 'temp_rolling_std_7'
]

TARGET_COL = 'temperature_celsius'

print(f"📊 Features: {len(FEATURE_COLS)}")
print(f"🎯 Target: {TARGET_COL}")

📊 Features: 24
🎯 Target: temperature_celsius


In [13]:
# Create country stats for inference
country_stats = daily_df.groupby('country').agg({
    'latitude': 'mean',
    'longitude': 'mean',
    'temperature_celsius': ['mean', 'std', 'min', 'max'],
    'country_encoded': 'first'
}).reset_index()

country_stats.columns = ['country', 'latitude', 'longitude', 'temp_mean', 'temp_std', 'temp_min', 'temp_max', 'country_encoded']

print("✅ Country stats created")
country_stats.head()

✅ Country stats created


,country,latitude,longitude,temp_mean,temp_std,temp_min,temp_max,country_encoded
0,Afghanistan,34.517341,69.182659,20.829496,10.244962,-3.3,36.3,0
1,Albania,41.327986,19.819114,21.133453,8.807284,1.3,39.2,1
2,Algeria,36.762502,3.050484,21.964324,6.912636,2.1,38.2,2
3,Andorra,42.500000,1.517341,10.941007,9.101585,-13.0,28.9,3
4,Angola,-8.838630,13.233545,26.044964,2.755828,19.3,32.2,4


---
## 4. Train/Test Split

In [14]:
# Time-based split (use last 20% of dates as test)
dates_sorted = sorted(daily_df['date'].unique())
split_date = dates_sorted[int(len(dates_sorted) * 0.8)]

train_df = daily_df[daily_df['date'] < split_date].copy()
test_df = daily_df[daily_df['date'] >= split_date].copy()

print(f"📅 Split date: {split_date}")
print(f"📊 Train: {len(train_df):,} rows ({len(train_df)/len(daily_df)*100:.1f}%)")
print(f"📊 Test: {len(test_df):,} rows ({len(test_df)/len(daily_df)*100:.1f}%)")

📅 Split date: 2025-09-04 00:00:00
📊 Train: 81,842 rows (79.7%)
📊 Test: 20,810 rows (20.3%)


In [15]:
# Prepare features and target
X_train = train_df[FEATURE_COLS].values
y_train = train_df[TARGET_COL].values
X_test = test_df[FEATURE_COLS].values
y_test = test_df[TARGET_COL].values

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"✅ Data prepared and scaled")
print(f"📊 X_train shape: {X_train_scaled.shape}")
print(f"📊 X_test shape: {X_test_scaled.shape}")

✅ Data prepared and scaled
📊 X_train shape: (81842, 24)
📊 X_test shape: (20810, 24)


In [16]:
# Convert to PyTorch tensors
X_train_tensor = torch.FloatTensor(X_train_scaled).to(device)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1).to(device)
X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
y_test_tensor = torch.FloatTensor(y_test).unsqueeze(1).to(device)

# Create DataLoaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

BATCH_SIZE = 256
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print(f"✅ DataLoaders created (batch_size={BATCH_SIZE})")

✅ DataLoaders created (batch_size=256)


---
## 5. Define MLP Model

In [17]:
class WeatherMLP(nn.Module):
    """
    Multi-Layer Perceptron for Weather Temperature Prediction.
    """
    def __init__(self, input_dim, hidden_dims=[256, 128, 64], dropout=0.3):
        super(WeatherMLP, self).__init__()
        
        layers = []
        prev_dim = input_dim
        
        for hidden_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = hidden_dim
        
        # Output layer
        layers.append(nn.Linear(prev_dim, 1))
        
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)

# Test model architecture
input_dim = len(FEATURE_COLS)
model = WeatherMLP(input_dim).to(device)
print(f"✅ Model created")
print(model)

✅ Model created
WeatherMLP(
  (network): Sequential(
    (0): Linear(in_features=24, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=256, out_features=128, bias=True)
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.3, inplace=False)
    (8): Linear(in_features=128, out_features=64, bias=True)
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.3, inplace=False)
    (12): Linear(in_features=64, out_features=1, bias=True)
  )
)


In [18]:
# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"📊 Total parameters: {total_params:,}")
print(f"📊 Trainable parameters: {trainable_params:,}")

📊 Total parameters: 48,513
📊 Trainable parameters: 48,513


---
## 6. Hyperparameter Tuning with Optuna

In [19]:
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    total_loss = 0
    for X_batch, y_batch in loader:
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

def evaluate(model, loader, criterion):
    model.eval()
    total_loss = 0
    all_preds = []
    all_targets = []
    with torch.no_grad():
        for X_batch, y_batch in loader:
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            total_loss += loss.item()
            all_preds.extend(y_pred.cpu().numpy().flatten())
            all_targets.extend(y_batch.cpu().numpy().flatten())
    
    mae = mean_absolute_error(all_targets, all_preds)
    return total_loss / len(loader), mae

In [20]:
def objective(trial):
    """Optuna objective function for hyperparameter tuning."""
    
    # Hyperparameters to tune
    n_layers = trial.suggest_int('n_layers', 2, 4)
    hidden_dims = [trial.suggest_int(f'hidden_dim_{i}', 64, 512) for i in range(n_layers)]
    dropout = trial.suggest_float('dropout', 0.1, 0.5)
    lr = trial.suggest_float('lr', 1e-4, 1e-2, log=True)
    weight_decay = trial.suggest_float('weight_decay', 1e-6, 1e-3, log=True)
    
    # Create model
    model = WeatherMLP(input_dim, hidden_dims, dropout).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)
    
    # Quick training (fewer epochs for tuning)
    n_epochs = 20
    best_mae = float('inf')
    
    for epoch in range(n_epochs):
        train_loss = train_epoch(model, train_loader, criterion, optimizer)
        val_loss, val_mae = evaluate(model, test_loader, criterion)
        
        if val_mae < best_mae:
            best_mae = val_mae
        
        # Early stopping for Optuna
        trial.report(val_mae, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
    
    return best_mae

In [21]:
# Run Optuna optimization
print("🔄 Running Optuna Hyperparameter Optimization...")
print("   This may take 5-10 minutes...")

study = optuna.create_study(direction='minimize', study_name='weather_mlp')
study.optimize(objective, n_trials=30, show_progress_bar=True)

print("\n" + "="*60)
print("✅ Optimization Complete!")
print(f"🏆 Best MAE: {study.best_value:.4f}°C")
print("\n📊 Best Hyperparameters:")
for key, value in study.best_params.items():
    print(f"   {key}: {value}")

🔄 Running Optuna Hyperparameter Optimization...
   This may take 5-10 minutes...


Best trial: 0. Best value: 1.55673: 100%|██████████| 30/30 [07:42<00:00, 15.42s/it]


✅ Optimization Complete!
🏆 Best MAE: 1.5567°C

📊 Best Hyperparameters:
   n_layers: 4
   hidden_dim_0: 359
   hidden_dim_1: 497
   hidden_dim_2: 345
   hidden_dim_3: 339
   dropout: 0.22427999020982847
   lr: 0.0001604836466303134
   weight_decay: 2.513632131586377e-06


---
## 7. Train Final Model with Best Hyperparameters

In [22]:
# Extract best hyperparameters
best_params = study.best_params
n_layers = best_params['n_layers']
hidden_dims = [best_params[f'hidden_dim_{i}'] for i in range(n_layers)]
dropout = best_params['dropout']
lr = best_params['lr']
weight_decay = best_params['weight_decay']

print(f"📊 Best architecture: {hidden_dims}")
print(f"📊 Dropout: {dropout:.3f}")
print(f"📊 Learning rate: {lr:.6f}")

📊 Best architecture: [359, 497, 345, 339]
📊 Dropout: 0.224
📊 Learning rate: 0.000160


In [23]:
# Create final model
final_model = WeatherMLP(input_dim, hidden_dims, dropout).to(device)
criterion = nn.MSELoss()
optimizer = optim.AdamW(final_model.parameters(), lr=lr, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

print("✅ Final model created")
print(final_model)

✅ Final model created
WeatherMLP(
  (network): Sequential(
    (0): Linear(in_features=24, out_features=359, bias=True)
    (1): BatchNorm1d(359, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.22427999020982847, inplace=False)
    (4): Linear(in_features=359, out_features=497, bias=True)
    (5): BatchNorm1d(497, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.22427999020982847, inplace=False)
    (8): Linear(in_features=497, out_features=345, bias=True)
    (9): BatchNorm1d(345, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): Dropout(p=0.22427999020982847, inplace=False)
    (12): Linear(in_features=345, out_features=339, bias=True)
    (13): BatchNorm1d(339, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): ReLU()
    (15): Dropout(p=0.22427999020982847, inplace=False)
    (16): Linear(in_features=339, out_featur

In [24]:
# Training loop with early stopping
N_EPOCHS = 100
PATIENCE = 15

best_mae = float('inf')
best_model_state = None
patience_counter = 0
history = {'train_loss': [], 'val_loss': [], 'val_mae': []}

print("🔥 Training final model...")
print("="*60)

for epoch in range(N_EPOCHS):
    train_loss = train_epoch(final_model, train_loader, criterion, optimizer)
    val_loss, val_mae = evaluate(final_model, test_loader, criterion)
    
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_mae'].append(val_mae)
    
    scheduler.step(val_mae)
    
    if val_mae < best_mae:
        best_mae = val_mae
        best_model_state = final_model.state_dict().copy()
        patience_counter = 0
        marker = '✓ Best'
    else:
        patience_counter += 1
        marker = ''
    
    if (epoch + 1) % 5 == 0 or marker:
        print(f"Epoch {epoch+1:3d}/{N_EPOCHS} | Train Loss: {train_loss:.4f} | Val MAE: {val_mae:.4f}°C {marker}")
    
    if patience_counter >= PATIENCE:
        print(f"\n⚠️ Early stopping at epoch {epoch+1}")
        break

# Load best model
final_model.load_state_dict(best_model_state)
print("\n" + "="*60)
print(f"✅ Training complete! Best MAE: {best_mae:.4f}°C")

🔥 Training final model...
Epoch   1/100 | Train Loss: 387.9364 | Val MAE: 13.8252°C ✓ Best
Epoch   2/100 | Train Loss: 141.2918 | Val MAE: 6.7597°C ✓ Best
Epoch   3/100 | Train Loss: 30.3736 | Val MAE: 2.3986°C ✓ Best
Epoch   4/100 | Train Loss: 10.5089 | Val MAE: 1.7346°C ✓ Best
Epoch   5/100 | Train Loss: 8.7382 | Val MAE: 1.6125°C ✓ Best
Epoch   6/100 | Train Loss: 8.3015 | Val MAE: 1.5870°C ✓ Best
Epoch  10/100 | Train Loss: 7.7700 | Val MAE: 1.6026°C 
Epoch  11/100 | Train Loss: 7.7326 | Val MAE: 1.5810°C ✓ Best
Epoch  12/100 | Train Loss: 7.6221 | Val MAE: 1.5740°C ✓ Best
Epoch  13/100 | Train Loss: 7.4910 | Val MAE: 1.5735°C ✓ Best
Epoch  15/100 | Train Loss: 7.3389 | Val MAE: 1.5696°C ✓ Best
Epoch  20/100 | Train Loss: 7.1313 | Val MAE: 1.5823°C 
Epoch  22/100 | Train Loss: 6.9579 | Val MAE: 1.5581°C ✓ Best
Epoch  25/100 | Train Loss: 6.9751 | Val MAE: 1.5705°C 
Epoch  30/100 | Train Loss: 6.8307 | Val MAE: 1.5606°C 
Epoch  35/100 | Train Loss: 6.9039 | Val MAE: 1.5878°C 

⚠️ E

In [25]:
# Plot training history
fig = make_subplots(rows=1, cols=2, subplot_titles=['Loss', 'MAE'])

fig.add_trace(go.Scatter(y=history['train_loss'], name='Train Loss', line=dict(color='#4ECDC4')), row=1, col=1)
fig.add_trace(go.Scatter(y=history['val_loss'], name='Val Loss', line=dict(color='#FF6B6B')), row=1, col=1)
fig.add_trace(go.Scatter(y=history['val_mae'], name='Val MAE', line=dict(color='#45B7D1')), row=1, col=2)

fig.update_layout(title='📈 Training History', height=400)
fig.show()

---
## 8. Evaluate Model

In [26]:
# Final evaluation
final_model.eval()
with torch.no_grad():
    y_pred = final_model(X_test_tensor).cpu().numpy().flatten()

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print("="*60)
print("📊 FINAL MODEL EVALUATION")
print("="*60)
print(f"   MAE:  {mae:.4f}°C")
print(f"   RMSE: {rmse:.4f}°C")
print(f"   R²:   {r2:.4f}")

📊 FINAL MODEL EVALUATION
   MAE:  1.5603°C
   RMSE: 2.2206°C
   R²:   0.9380


In [27]:
# Actual vs Predicted scatter plot
fig = px.scatter(
    x=y_test, y=y_pred,
    labels={'x': 'Actual Temperature (°C)', 'y': 'Predicted Temperature (°C)'},
    title=f'🎯 Actual vs Predicted (R² = {r2:.4f})',
    opacity=0.3
)
fig.add_trace(go.Scatter(x=[-20, 50], y=[-20, 50], mode='lines', name='Perfect', line=dict(color='red', dash='dash')))
fig.update_layout(height=500)
fig.show()

In [28]:
# Error distribution
errors = y_pred - y_test

fig = px.histogram(
    x=errors, nbins=50,
    title='📊 Prediction Error Distribution',
    labels={'x': 'Error (°C)', 'y': 'Count'}
)
fig.add_vline(x=0, line_dash='dash', line_color='red')
fig.update_traces(marker_color='#4ECDC4')
fig.update_layout(height=400)
fig.show()

print(f"📊 Error Stats:")
print(f"   Mean Error: {np.mean(errors):.4f}°C")
print(f"   Std Error:  {np.std(errors):.4f}°C")
print(f"   95% of errors within: ±{np.percentile(np.abs(errors), 95):.2f}°C")

📊 Error Stats:
   Mean Error: 0.1948°C
   Std Error:  2.2120°C
   95% of errors within: ±4.65°C


In [29]:
# Evaluate per country (sample)
test_df['prediction'] = y_pred
test_df['error'] = test_df['prediction'] - test_df['temperature_celsius']

country_mae = test_df.groupby('country')['error'].apply(lambda x: np.abs(x).mean()).sort_values()

print("\n🌍 TOP 10 BEST COUNTRIES (Lowest MAE):")
print(country_mae.head(10).to_string())

print("\n🌍 TOP 10 WORST COUNTRIES (Highest MAE):")
print(country_mae.tail(10).to_string())


🌍 TOP 10 BEST COUNTRIES (Lowest MAE):
country
Kiribati       0.290092
Dominica       0.304956
Maldives       0.344510
Peru           0.406210
Somalia        0.473603
Ghana          0.536037
Suriname       0.546570
Liberia        0.554118
East Timor     0.589449
Ivory Coast    0.591954

🌍 TOP 10 WORST COUNTRIES (Highest MAE):
country
Switzerland    2.678929
China          2.693803
Kyrgyzstan     2.714156
Venezuela      2.912203
Eswatini       3.038591
Kazakhstan     3.050525
Slovenia       3.160985
Mongolia       3.223324
Australia      3.311844
Canada         3.892543


---
## 9. Save Model Artifacts

In [30]:
# Create models directory
os.makedirs('../models', exist_ok=True)

# Save PyTorch model
torch.save({
    'model_state_dict': final_model.state_dict(),
    'input_dim': input_dim,
    'hidden_dims': hidden_dims,
    'dropout': dropout
}, '../models/global_weather_mlp.pt')
print("✅ Model saved: models/global_weather_mlp.pt")

# Save scaler
joblib.dump(scaler, '../models/feature_scaler.joblib')
print("✅ Scaler saved: models/feature_scaler.joblib")

# Save country encoder
joblib.dump(country_encoder, '../models/country_encoder.joblib')
print("✅ Country encoder saved: models/country_encoder.joblib")

# Save country stats
country_stats.to_csv('../models/country_stats.csv', index=False)
print("✅ Country stats saved: models/country_stats.csv")

✅ Model saved: models/global_weather_mlp.pt
✅ Scaler saved: models/feature_scaler.joblib
✅ Country encoder saved: models/country_encoder.joblib
✅ Country stats saved: models/country_stats.csv


In [31]:
# Save model config
config = {
    'feature_columns': FEATURE_COLS,
    'target_column': TARGET_COL,
    'input_dim': input_dim,
    'hidden_dims': hidden_dims,
    'dropout': dropout,
    'n_countries': len(country_encoder.classes_),
    'metrics': {
        'mae': float(mae),
        'rmse': float(rmse),
        'r2': float(r2)
    },
    'training': {
        'n_epochs': len(history['train_loss']),
        'best_epoch': int(np.argmin(history['val_mae']) + 1),
        'train_samples': len(train_df),
        'test_samples': len(test_df)
    }
}

with open('../models/model_config.json', 'w') as f:
    json.dump(config, f, indent=2)
print("✅ Config saved: models/model_config.json")

✅ Config saved: models/model_config.json


In [32]:
# Save cleaned data for reference
daily_df.to_csv('../data/processed/weather_cleaned.csv', index=False)
print("✅ Cleaned data saved: data/processed/weather_cleaned.csv")

✅ Cleaned data saved: data/processed/weather_cleaned.csv


---
## 10. Demo: 7-Day Forecast

In [33]:
def predict_7day_forecast(model, country, start_date, country_encoder, scaler, country_stats, feature_cols, device):
    """
    Generate 7-day temperature forecast for a country.
    """
    model.eval()
    
    # Get country info
    stats = country_stats[country_stats['country'] == country].iloc[0]
    country_encoded = int(stats['country_encoded'])
    lat = stats['latitude']
    lon = stats['longitude']
    
    # Initialize with country's average temperature for lag features
    temp_history = [stats['temp_mean']] * 30
    
    forecasts = []
    current_date = pd.to_datetime(start_date)
    
    for day in range(7):
        # Create features
        features = {
            'country_encoded': country_encoded,
            'latitude': lat,
            'longitude': lon,
            'month': current_date.month,
            'day_of_month': current_date.day,
            'day_of_week': current_date.dayofweek,
            'day_of_year': current_date.dayofyear,
            'quarter': (current_date.month - 1) // 3 + 1,
            'is_weekend': 1 if current_date.dayofweek >= 5 else 0,
            'month_sin': np.sin(2 * np.pi * current_date.month / 12),
            'month_cos': np.cos(2 * np.pi * current_date.month / 12),
            'day_sin': np.sin(2 * np.pi * current_date.dayofweek / 7),
            'day_cos': np.cos(2 * np.pi * current_date.dayofweek / 7),
            'day_of_year_sin': np.sin(2 * np.pi * current_date.dayofyear / 365),
            'day_of_year_cos': np.cos(2 * np.pi * current_date.dayofyear / 365),
            'temp_lag_1': temp_history[-1],
            'temp_lag_2': temp_history[-2],
            'temp_lag_3': temp_history[-3],
            'temp_lag_7': temp_history[-7],
            'temp_lag_14': temp_history[-14],
            'temp_lag_30': temp_history[-30],
            'temp_rolling_mean_7': np.mean(temp_history[-7:]),
            'temp_rolling_mean_14': np.mean(temp_history[-14:]),
            'temp_rolling_std_7': np.std(temp_history[-7:])
        }
        
        # Create feature vector
        X = np.array([[features[col] for col in feature_cols]])
        X_scaled = scaler.transform(X)
        X_tensor = torch.FloatTensor(X_scaled).to(device)
        
        # Predict
        with torch.no_grad():
            pred = model(X_tensor).cpu().item()
        
        forecasts.append({
            'date': current_date.strftime('%Y-%m-%d'),
            'day_name': current_date.strftime('%A'),
            'temperature': round(pred, 1)
        })
        
        # Update history
        temp_history.append(pred)
        current_date += timedelta(days=1)
    
    return forecasts

In [34]:
# Demo forecast
DEMO_COUNTRY = 'Egypt'
DEMO_DATE = '2025-01-15'

forecast = predict_7day_forecast(
    final_model, DEMO_COUNTRY, DEMO_DATE,
    country_encoder, scaler, country_stats, FEATURE_COLS, device
)

print(f"\n🌍 7-Day Weather Forecast for {DEMO_COUNTRY}")
print(f"📅 Starting: {DEMO_DATE}")
print("="*50)
for day in forecast:
    print(f"   {day['date']} ({day['day_name']:9s}): {day['temperature']:5.1f}°C")

# Summary
temps = [d['temperature'] for d in forecast]
print("\n📊 Summary:")
print(f"   Min: {min(temps):.1f}°C")
print(f"   Max: {max(temps):.1f}°C")
print(f"   Avg: {np.mean(temps):.1f}°C")


🌍 7-Day Weather Forecast for Egypt
📅 Starting: 2025-01-15
   2025-01-15 (Wednesday):  27.1°C
   2025-01-16 (Thursday ):  27.2°C
   2025-01-17 (Friday   ):  27.1°C
   2025-01-18 (Saturday ):  27.1°C
   2025-01-19 (Sunday   ):  27.0°C
   2025-01-20 (Monday   ):  27.2°C
   2025-01-21 (Tuesday  ):  27.5°C

📊 Summary:
   Min: 27.0°C
   Max: 27.5°C
   Avg: 27.2°C


In [35]:
# Visualize forecast
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=[d['date'] for d in forecast],
    y=[d['temperature'] for d in forecast],
    mode='lines+markers',
    name='Temperature',
    line=dict(color='#FF6B6B', width=3),
    marker=dict(size=10)
))

fig.update_layout(
    title=f'🌡️ 7-Day Temperature Forecast for {DEMO_COUNTRY}',
    xaxis_title='Date',
    yaxis_title='Temperature (°C)',
    height=400
)
fig.show()

---
## 📊 Summary

### What We Built:
- **Single MLP model** trained on all countries
- **20 input features** (temporal, cyclical, geographic, lag)
- **1 output** (temperature_celsius)

### Artifacts Saved:
- `models/global_weather_mlp.pt` - PyTorch model
- `models/feature_scaler.joblib` - StandardScaler
- `models/country_encoder.joblib` - LabelEncoder
- `models/country_stats.csv` - Country metadata
- `models/model_config.json` - Configuration

### Next Steps:
- Build FastAPI web application
- Create interactive frontend
- Deploy to production